In [1]:
from glob import glob
import json
import re
import yaml
import os
os.chdir('../')

In [2]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [3]:
queues_cfg = cfg['QUEUES']

In [11]:
tweets_queue_path = queues_cfg['new_tweets']
sa_queue_path = queues_cfg['sentiment_tasks']
geo_queue_path = queues_cfg['geo_tasks']

In [16]:
def create_geo_task(tweet_id, coordinates):
    filename = '{}/{}.task.txt'.format(geo_queue_path, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(json.dumps(coordinates))
        fp.close()

def create_sentiment_task(tweet_id, text):
    filename = '{}/{}.task.txt'.format(sa_queue_path, tweet_id)
    with open(filename, 'w') as fp:
        fp.write(text)
        fp.close()

In [17]:
unprocessed_tweets = glob('{}/*.json'.format(tweets_queue_path))
len(unprocessed_tweets)

44166

pathes = glob('../data/tweet_no_coordinates/*.json')
len(pathes)

In [18]:
hashtags = ['bitcoin', 'blockchain', 'btc', 'cryptocurrency', 'market\svalue',
            'crypto','ethereum', 'fintech', 'coin', 'doge', 'ethereum', 'ripple', 'litecoin', 
            'cardano', 'monero', 'TRON', 'zcash', 'jaxx', 'copay', 'bitpay', 'exodus', 
            'mycelium', 'Bread\sWallet', 'trezor', 'ledger\snano', 'Silk\sRoad', 'darknet\smarket', 
            'dogecoin', 'ASIC\sMiner', 'Central\sLedger', 'Hashrate', 'ICO',         
           ]

In [19]:
'|'.join(hashtags)

'bitcoin|blockchain|btc|cryptocurrency|market\\svalue|crypto|ethereum|fintech|coin|doge|ethereum|ripple|litecoin|cardano|monero|TRON|zcash|jaxx|copay|bitpay|exodus|mycelium|Bread\\sWallet|trezor|ledger\\snano|Silk\\sRoad|darknet\\smarket|dogecoin|ASIC\\sMiner|Central\\sLedger|Hashrate|ICO'

In [20]:
bitcoin_pathes = []
for path in (unprocessed_tweets):
    with open(path, 'r') as fp:
        data = json.load(fp)
        if re.search('|'.join(hashtags), json.dumps(data)):
            bitcoin_pathes.append(path)
        #print(data['entities']['hashtags'])


In [21]:
len(bitcoin_pathes)

384

In [29]:
invalid_path = ''
for path in bitcoin_pathes:
    with open(path, 'r') as fp:
        data = json.load(fp)
        tweet_id = data['id_str']
        coordinates = []
        
        try:
            if data['coordinates'] is not None:
                coordinates = data['coordinates']
            elif data['place'] is not None:
                coordinates = data['place']['bounding_box']
            else:
                print(tweet_id)
            create_geo_task(tweet_id, coordinates=coordinates)
        except Exception as e:
            print(tweet_id)
            print(data)
            invalid_path = path
            raise e
            
        #print(coordinates)
        

988986510979497985


In [28]:
with open(invalid_path, 'r') as fp:
    data = json.load(fp)   
data

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Apr 25 03:42:13 +0000 2018',
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/988986510979497985',
    'indices': [94, 117],
    'url': 'https://t.co/JBN4A0CmUs'}],
  'user_mentions': []},
 'extended_tweet': {'display_text_range': [0, 221],
  'entities': {'hashtags': [],
   'media': [{'display_url': 'pic.twitter.com/SP3y2HBVVt',
     'expanded_url': 'https://twitter.com/sarahfisher6333/status/988986510979497985/photo/1',
     'id': 988986505392685056,
     'id_str': '988986505392685056',
     'indices': [222, 245],
     'media_url': 'http://pbs.twimg.com/media/DbmV_J2WAAAGiZx.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/DbmV_J2WAAAGiZx.jpg',
     'sizes': {'large': {'h': 1096, 'resize': 'fit', 'w': 2048},
      'medium': {'h': 642, 'resize': 'fit', 'w': 1200},
      '